# 8.1 LSTM으로 텍스트 생성하기

## 8.1.2 시퀀스 데이터를 어떻게 생성할까?

딥러닝에서 시퀀스를 생성하는 일반적인 방법은 이전 토큰들이 주어졌을 때 다음 토큰의 확률을 모델링하는 Language Model.

## 8.1.3 샘플링 전략의 중요성

텍스트를 생성할 때 다음 글자를 선택하는 방법들.

- Greedy Sampling : 단순히 항상 가장 높은 확률을 가진 글자를 선택 -> 논리적인 언어로 보이지 않음
- Stochastic Sampling : 다음 글자를 확률적으로 선택(e가 0.3의 확률이면 10번 중 3번은 e 선택)

## 8.1.4 글자 수준의 LSTM 텍스트 생성 모델 구현

In [1]:
# 데이터 전처리

import keras
import numpy as np

path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()

print('말뭉치 크기:', len(text))

600901/600901 [==============================] - 1s 2us/step
말뭉치 크기: 600893


In [3]:
# 글자 시퀀스 벡터화

maxlen = 60
step = 3

sentences=[]
next_chars = []     # 타깃(다음 글자)를 담을 리스트

for i in range(0, len(text)-maxlen, step):
    sentences.append(text[i: i+maxlen])
    next_chars.append(text[i + maxlen])
    
print('시퀀스 개수:', len(sentences))

chars = sorted(list(set(text)))
print('고유한 글자:', len(chars))

char_indices = dict((char, chars.index(char)) for char in chars)

print('벡터화...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

시퀀스 개수: 200278
고유한 글자: 58
벡터화...


C:\Users\ytnal\AppData\Local\Temp\ipykernel_15108\3939754824.py:21: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
C:\Users\ytnal\AppData\Local\Temp\ipykernel_15108\3939754824.py:22: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool

In [4]:
# 네트워크 구성

from keras import layers

model = keras.models.Sequential([
    layers.LSTM(128, input_shape=(maxlen, len(chars))),
    layers.Dense(len(chars), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=0.01))

C:\Users\ytnal\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [5]:
# 모델의 예측이 주어졌을 때 새로운 글자를 샘플링하는 함수
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
# 텍스트 생성 루프

import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

for epoch in range(1, 60):     # 60에포크동안 훈련
    print('에포크', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    
    seed_text = text[start_index: start_index +  maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.          # 지금까지 생성된 글자를 원핫인코딩으로 바꿈
            
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
1565/1565 [==============================] - 115s 72ms/step - loss: 1.9499
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the senser of the soult the soult of the so the sense that the so the soult of the so the same and which the soult that the which the soul and the soult and the so the soul and the so the many in the strong and which the man to a sake of the most the soul and such a so the so the stringer and life the strong to the soult and all the soult to the strong of the same to the soult that the strenge t
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through for the cause and the could to a being that the from the lattle and
compless of the while of the exprinding so that the as in thesely hould that the senser sen and he who success of have not men the which the manser and so the ferthers and so the strough that the acted that and moough that the 

KeyboardInterrupt: 

시드 텍스트 'the slowly ascending ranks and classes, inwhice, through fo'을 무작위로 시드텍스트로 선택한 뒤 temperature에 따라 생성

# 8.2~3

그 이외에 이미지를 비슷하게 예술적으로 변경하는 딥드림, 뉴럴스타일 트랜스퍼 등등 존재. 코드는 쓰고싶으면 알아서

# 8.4 변이형 오토인코더를 사용한 이미지 생성

- 변이형 오토인코더(VAE) : 이미지의 잠재 공간에서 샘플링해서 완전히 새로운 이미지나 기존 이미지를 변형하는 방식

## 8.4.1 이미지의 잠재 공간에서 샘플링하기

잠재공간의 한 포인트를 입력으로 받아 이미지를 출력하는 모듈은 생성자(GAN에서) or 디코더(VAE)라고 부름

잠재공간이 만들어지면 여기서 포인트 하나를 특정/무작위 샘플링을 통해 이미지 생성 가능

## VAE의 작동방식

1. 인코더 모듈이 입력 샘플 input_img를 잠재 공간의 두 파라미터 z_mean과 z_log_var로 변환
2. 입력 이미지가 생성되었다고 가정한 잠재공간의 정규분포에서 포인트 z를 z = z_mean + exp(0.5 * z_log_var) * epsilon 처럼 무작위로 샘플링
3. 디코더 모듈은 잠재 공간의 이 포인트를 원본 입력 이미지르 매핑하여 복원

VAE는 2개의 손실함수 존재
- 재구성 손실 : 디코딩된 샘플이 원본 입력과 동일하도록 만듬
- 규제 손실 : 잠재 공간을 잘 형성하고 훈련 데이터에 과대적합을 줄임

In [19]:
# VAE 인코더 네트워크
import keras
from keras import layers
from keras import backend as K
from keras.models import Model
import numpy as np

img_shape = (28, 28, 1)
batch_size=16
latent_dim=2

input_img = keras.Input(shape=img_shape)

x = layers.Conv2D(32, 3, padding='same', activation='relu')(input_img)
x = layers.Conv2D(64, 3, padding='same', activation='relu', strides=(2, 2))(x)
x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)

shape_before_flattening = K.int_shape(x)

x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)

z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

In [21]:
# 잠재공간 샘플링함수
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape = (K.shape(z_mean)[0], latent_dim), mean=0, stddev=1)
    
    return z_mean, K.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

In [22]:
# VAE 디코더 네트워크

decoder_input = layers.Input(K.int_shape(z)[1:])

x = layers.Dense(np.prod(shape_before_flattening[1:]), activation='relu')(decoder_input)
x = layers.Reshape(shape_before_flattening[1:])(x)
x = layers.Conv2DTranspose(32, 3, padding='same', activation='relu', strides=(2, 2))(x)

decoder = Model(decoder_input, x)
z_decoded = decoder(z)

AttributeError: 'tuple' object has no attribute 'shape'

# 8.5 적대적 생성 신경망(GAN)

GAN은 위조 이미지를 생성하는 생성자와 그를 바탕으로 진위여부를 판별하는 판별자로 구분. 판별자가 진짜라고 판정할 때까지 반복해서 생성

## 8.5.3 생성자

In [23]:
import keras
from keras import layers
import numpy as np

latent_dim=32
height=32
width=32
channels=3

generator_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense_4 (Dense)             (None, 32768)             1081344   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32768)             0         
                                                                 
 reshape (Reshape)           (None, 16, 16, 128)       0         
                                                                 
 conv2d_286 (Conv2D)         (None, 16, 16, 256)       819456    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 16, 16, 256)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 32, 32, 256)      104883

## 8.5.4 판별자

In [24]:
discriminator_input = layers.Input(shape=(height, width, channels))

x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)

x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_290 (Conv2D)         (None, 30, 30, 128)       3584      
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 30, 30, 128)       0         
                                                                 
 conv2d_291 (Conv2D)         (None, 14, 14, 128)       262272    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 14, 14, 128)       0         
                                                                 
 conv2d_292 (Conv2D)         (None, 6, 6, 128)         262272    
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 6, 6, 128)         0   

C:\Users\ytnal\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


## 8.5.5 적대적 네트워크

In [25]:
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

## 8.5.6 DCGAN 훈련방법

1. 잠재공간에서 무작위로 포인트를 뽑음(랜덤 노이즈)
2. 이 랜덤 노이즈를 사용하여 generator에서 이미지를 생성
3. 생성된 이미지와 진짜 이미지를 섞음
4. 진짜와 가짜가 섞인 이미지와 이에 대응하는 타기을 사용하여 discriminator를 훈련
5. 잠재 공간에서 무작위로 새로운 포인트를 뽑음
6. 이 랜덤 벡터를 사용하여 gan을 훈련. 모든 타깃은 '진짜'로 설정. 즉 생성자는 판별자를 속이도록 훈련

In [31]:
import os
from keras.utils import array_to_img

(X_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

X_train = X_train[y_train.flatten() == 6]         # 클래스가 6인 데이터만 선택

X_train = X_train.reshape((X_train.shape[0],) + (height, width, channels)).astype('float32') / 255.     # 데이터 정규화

iterations = 10000
batch_size = 20
save_dir = 'C:/Users/ytnal/python/케라스창시자/datasets/gan_images/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
    
start = 0

for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))   # 잠재공간에서 무작위로 샘플링
    
    generated_images = generator.predict(random_latent_vectors)    # 가짜 이미지를 디코딩
    
    stop = start + batch_size
    real_images = X_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])
    
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])  # 진짜와 가짜를 구분하여 레이블을 합침
    labels += 0.05 * np.random.random(labels.shape)   # 레이블에 랜덤 노이즈를 추가
    
    d_loss = discriminator.train_on_batch(combined_images, labels)   # 판별자 훈련
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    misleading_targets = np.zeros((batch_size, 1))     # 모두 진짜 이미지라고 레이블 생성
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    
    if start > len(X_train) - batch_size:
        start = 0
    if step % 100 == 0:     # 100스텝마다 중간중간 저장
        gan.save_weights('gan.h5')
        
        print('판별자 손실:', d_loss)
        print('적대적 손실:', a_loss)
        
        img = array_to_img(generated_images[0] * 255, scale=False)
        img.save(os.path.join(save_dir, 'generated_frog' + str(step) + '.png'))
        
        img = array_to_img(real_images[0] * 255, scale=False)
        img.save(os.path.join(save_dir, 'real_frog' + str(step) + '.png'))

판별자 손실: 1.6427397
적대적 손실: 0.0006525054


KeyboardInterrupt: 